In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=4629c77f217834202bd48ccabd979c9b565d63648eb7b6f7e882b54f9ebf3ff4
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 15.36 ,random_state=0 , l1_ratio=0.9898, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 58.29 , C = 977.21 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_204000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_20_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=916566, Mon Apr 13 17:59:14 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.983052776632538e+06 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:14 2020)
final/bestever f-value = 1.983053e+06 1.983053e+06
incumbent solution: [0.82335117 0.96079761 0.28052882 0.66223285 0.4344831  0.48859487
 0.01672487 0.40141392 ...]
std deviations: [0.15058536 0.15053426 0.15051042 0.15053796 0.15048164 0.15043203
 0.15065967 0.15055335 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=944576, Mon Apr 13 17:59:14 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.969341128382152e+06 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1983052.77663254] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1983052.776633, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1983052.776633, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=

termination on tolfun=1e-11 (Mon Apr 13 17:59:14 2020)
final/bestever f-value = 1.969341e+06 1.969341e+06
incumbent solution: [ 0.76060734  0.92171003  0.31786458  0.69340092  0.36446435  0.58617883
 -0.01960466  0.44547223 ...]
std deviations: [0.15058728 0.1505133  0.15055719 0.15054629 0.15060131 0.15057867
 0.15062489 0.15056177 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=947418, Mon Apr 13 17:59:14 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.976919709813358e+06 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 1.952836327676133e+06 1.0e+00 1.47e-01  1e-01  1e-01 0:00.1
    3     51 1.922891552297239e+06 1.0e+00 1.44e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1998081.05202834] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1952836.32767613] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   77   1309 -1.308078309176140e+08 1.3e+00 1.90e+00  2e+00  2e+00 0:03.2
  100   1700 -1.034959943020192e+09 1.3e+00 5.23e+00  5e+00  5e+00 0:04.1
  169   2873 -6.362808874732544e+11 1.3e+00 1.59e+02  2e+02  2e+02 0:06.9
termination on tolfacupx=1000.0 (Mon Apr 13 17:59:22 2020)
final/bestever f-value = -6.219913e+11 -6.362809e+11
incumbent solution: [-407.9361694   794.00294742  312.57044435  277.04644205  319.65261817
 -603.79398633  530.16066944  -12.69084301 ...]
std deviations: [156.97517276 156.25287755 157.18948144 157.45873765 157.4171304
 157.86814298 156.81751583 156.4783065  ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=934418, Mon Apr 13 17:59:22 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.983052776632538e+06 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:22 2020)
final/bestever f-value = 1.983053e+06 1.983053e+06
incumbent solution: [0.03657854 0.51313547 0.8805

...

   76   1292 -8.996562731014843e+07 1.3e+00 1.50e+00  1e+00  2e+00 0:03.1
  100   1700 -7.020912497481759e+08 1.3e+00 4.34e+00  4e+00  4e+00 0:04.1
  181   3077 -1.113840412013355e+12 1.3e+00 1.61e+02  2e+02  2e+02 0:07.5
termination on tolfacupx=1000.0 (Mon Apr 13 18:03:20 2020)
final/bestever f-value = -1.079989e+12 -1.113840e+12
incumbent solution: [-596.48365187  169.88882586  -94.31151514  237.29948526  718.1925637
 -685.05299009  362.26220307    4.44360776 ...]
std deviations: [160.67869465 159.1246587  159.00901734 157.85539608 159.15478511
 160.22866547 159.68763888 157.69155982 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=914486, Mon Apr 13 18:03:20 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 1.983052776632538e+06 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 18:03:20 2020)
final/bestever f-value = 1.983053e+06 1.983053e+06
incumbent solution: [-0.10709512  0.76469125  0.

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [1983052.77663254] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1983052.776633, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=1983052.776633, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=

termination on tolfun=1e-11 (Mon Apr 13 18:03:20 2020)
final/bestever f-value = 1.969341e+06 1.969341e+06
incumbent solution: [0.18456101 0.84979213 0.78687199 0.8939863  0.26353543 0.14854616
 0.36482677 0.61778675 ...]
std deviations: [0.15068431 0.15056976 0.15046367 0.15052783 0.15049936 0.15054895
 0.15059367 0.1505653  ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=913336, Mon Apr 13 18:03:20 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.059918191882855e+06 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
    2     34 2.011567545125672e+06 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 1.962744349700416e+06 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2137456.82757015] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [2065880.48154223] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   76   1292 -2.248561812508322e+08 1.4e+00 2.37e+00  2e+00  2e+00 0:03.1
  100   1700 -2.031766998191999e+09 1.4e+00 7.36e+00  7e+00  7e+00 0:04.1
  166   2822 -1.010010441276052e+12 1.4e+00 1.55e+02  2e+02  2e+02 0:06.8
termination on tolfacupx=1000.0 (Mon Apr 13 18:03:28 2020)
final/bestever f-value = -9.671591e+11 -1.010010e+12
incumbent solution: [-709.86786975 -270.58985944   21.44513103  147.08326057  328.56455768
 -371.74800047  124.38784833   72.37449194 ...]
std deviations: [153.29688507 153.51352004 154.00375258 154.00405183 153.88321646
 155.12930045 154.1281152  153.45615027 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F20_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:100])
    SVM_Fun [i] = F20(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F20(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([97625.62547969]), count=array([1])) 15468.356977884707
SVM
ModeResult(mode=array([82905.06780174]), count=array([1])) 17943.176842532357
ELN
ModeResult(mode=array([3.13891209e+10]), count=array([1])) 5129980438.914328
